# Embryo Data Query

This notebook queries embryo-related data from multiple databases for a given prontuario/PIN.

## Tables Queried:
- **Clinisys Silver Layer** (clinisys_all.duckdb):
  - view_micromanipulacao
  - view_micromanipulacao_oocitos
  - view_tratamentos
  - view_congelamentos_embrioes
  - view_descongelamentos_embrioes
  - view_embrioes_congelados
- **Huntington Data Lake** (huntington_data_lake.duckdb):
  - gold.clinisys_embrioes
  - gold.embryoscope_embrioes
  - gold.embryoscope_clinisys_combined
  - gold.embryoscope_treatment (from embryoscope_embrioes)
  - silver.planilha_embriologia
  - gold.planilha_embryoscope_combined


In [32]:
import duckdb
import pandas as pd
import os
from datetime import datetime

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


## Database Connections


In [33]:
def get_clinisys_connection():
    """Get connection to clinisys_all database"""
    clinisys_db_path = os.path.join('..', '..', 'database', 'clinisys_all.duckdb')
    return duckdb.connect(clinisys_db_path, read_only=True)

def get_huntington_connection():
    """Get connection to huntington_data_lake database"""
    huntington_db_path = os.path.join('..', '..', 'database', 'huntington_data_lake.duckdb')
    return duckdb.connect(huntington_db_path, read_only=True)


## Query Functions - Clinisys Tables


In [34]:
def query_micromanipulacao(conn, prontuario):
    """Query view_micromanipulacao for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO ===")
    query = f"""
    SELECT * FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    ORDER BY codigo_ficha DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_micromanipulacao")
    return df

def query_micromanipulacao_oocitos(conn, prontuario):
    """Query view_micromanipulacao_oocitos for a specific prontuario"""
    print(f"\n=== VIEW_MICROMANIPULACAO_OOCITOS ===")
    
    # First get codigo_ficha from view_micromanipulacao for this prontuario
    codigos_query = f"""
    SELECT codigo_ficha FROM silver.view_micromanipulacao 
    WHERE prontuario = {prontuario}
    """
    
    codigos_df = conn.execute(codigos_query).df()
    if not codigos_df.empty:
        codigo_list = codigos_df['codigo_ficha'].tolist()
        placeholders = ','.join(['?'] * len(codigo_list))
        
        oocitos_query = f"""
        SELECT * FROM silver.view_micromanipulacao_oocitos
        WHERE id_micromanipulacao IN ({placeholders})
        ORDER BY id_micromanipulacao, id
        """
        
        df = conn.execute(oocitos_query, codigo_list).df()
        print(f"Found {len(df)} records in view_micromanipulacao_oocitos")
    else:
        print("No codigo_ficha found for this prontuario in view_micromanipulacao.")
        df = pd.DataFrame()
    
    return df


In [35]:
def query_tratamentos(conn, prontuario):
    """Query view_tratamentos for a specific prontuario"""
    print(f"\n=== VIEW_TRATAMENTOS ===")
    query = f"""
    SELECT * FROM silver.view_tratamentos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_tratamentos")
    return df

def query_congelamentos_embrioes(conn, prontuario):
    """Query view_congelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_embrioes")
    return df

def query_descongelamentos_embrioes(conn, prontuario):
    """Query view_descongelamentos_embrioes for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_EMBRIOES ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_embrioes")
    return df

def query_embrioes_congelados(conn, prontuario):
    """Query view_embrioes_congelados for a specific prontuario"""
    print(f"\n=== VIEW_EMBRIOES_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_embrioes_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_embrioes_congelados")
    return df

def query_congelamentos_ovulos(conn, prontuario):
    """Query view_congelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_CONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_congelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_congelamentos_ovulos")
    return df

def query_descongelamentos_ovulos(conn, prontuario):
    """Query view_descongelamentos_ovulos for a specific prontuario"""
    print(f"\n=== VIEW_DESCONGELAMENTOS_OVULOS ===")
    query = f"""
    SELECT * FROM silver.view_descongelamentos_ovulos 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_descongelamentos_ovulos")
    return df

def query_ovulos_congelados(conn, prontuario):
    """Query view_ovulos_congelados for a specific prontuario"""
    print(f"\n=== VIEW_OVULOS_CONGELADOS ===")
    query = f"""
    SELECT * FROM silver.view_ovulos_congelados 
    WHERE prontuario = {prontuario}
    ORDER BY id DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in view_ovulos_congelados")
    return df


## Query Functions - Huntington Data Lake Tables


In [36]:
def query_clinisys_embrioes(conn, prontuario):
    """Query gold.clinisys_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.CLINISYS_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.clinisys_embrioes 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, oocito_id_micromanipulacao DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.clinisys_embrioes")
    return df

def query_embryoscope_embrioes(conn, prontuario):
    """Query gold.embryoscope_embrioes for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_EMBRIOES ===")
    query = f"""
    SELECT * FROM gold.embryoscope_embrioes 
    WHERE prontuario = {prontuario}
    ORDER BY embryo_KIDDate DESC, treatment_TreatmentName DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_embrioes")
    return df

def query_embryoscope_clinisys_combined(conn, prontuario):
    """Query gold.embryoscope_clinisys_combined for a specific prontuario"""
    print(f"\n=== GOLD.EMBRYOSCOPE_CLINISYS_COMBINED ===")
    query = f"""
    SELECT * FROM gold.embryoscope_clinisys_combined 
    WHERE micro_prontuario = {prontuario}
    ORDER BY micro_Data_DL DESC, oocito_id_micromanipulacao DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.embryoscope_clinisys_combined")
    return df


In [37]:
def query_planilha_embriologia_fresh(conn, prontuario):
    """Query silver.planilha_embriologia for a specific prontuario"""
    print(f"\n=== SILVER.PLANILHA_EMBRIOLOGIA_FRESH ===")
    query = f"""
    SELECT * FROM silver.planilha_embriologia_fresh 
    WHERE prontuario = {prontuario}
    -- ORDER BY "DATA DA PUNÇÃO" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.planilha_embriologia_fresh")
    return df

def query_planilha_embriologia_fet(conn, prontuario):
    """Query silver.planilha_embriologia for a specific prontuario"""
    print(f"\n=== SILVER.PLANILHA_EMBRIOLOGIA_FET ===")
    query = f"""
    SELECT * FROM silver.planilha_embriologia_fet 
    WHERE prontuario = {prontuario}
    -- ORDER BY "DATA DA FET" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.planilha_embriologia_fresh")
    return df

def query_planilha_embryoscope_combined(conn, prontuario):
    """Query gold.planilha_embryoscope_combined for a specific prontuario"""
    print(f"\n=== GOLD.PLANILHA_EMBRYOSCOPE_COMBINED ===")
    query = f"""
    SELECT * FROM gold.planilha_embryoscope_combined 
    WHERE "micro_prontuario" = {prontuario}
    ORDER BY "oocito_id" DESC
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.planilha_embryoscope_combined")
    return df


In [38]:
def query_redlara_unified(conn, prontuario):
    """Query silver.redlara_unified for a specific prontuario"""
    print(f"\n=== silver.redlara_unified ===")
    query = f"""
    SELECT * FROM silver.redlara_unified
    where prontuario = {prontuario}
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in silver.redlara_unified")
    return df


def query_redlara_planilha_combined(conn, prontuario):
    """Query gold.redlara_planilha_combined for a specific prontuario"""
    print(f"\n=== gold.redlara_planilha_combined ===")
    query = f"""
    SELECT * FROM gold.redlara_planilha_combined
    where prontuario = {prontuario}
    """
    
    df = conn.execute(query).df()
    print(f"Found {len(df)} records in gold.redlara_planilha_combined")
    return df


## Main Query Function


In [39]:
def query_embryo_data(prontuario):
    """
    Query all embryo-related data for a given prontuario/PIN.
    
    Args:
        prontuario (str or int): The patient prontuario number
        
    Returns:
        dict: Dictionary containing all DataFrames with results
    """
    print(f"\n{'='*80}")
    print(f"QUERYING EMBRYO DATA FOR PRONTUARIO: {prontuario}")
    print(f"Query executed at: {datetime.now()}")
    print(f"{'='*80}")
    
    # Connect to databases
    clinisys_con = get_clinisys_connection()
    huntington_con = get_huntington_connection()
    
    try:
        # Query clinisys tables
        df_micromanipulacao = query_micromanipulacao(clinisys_con, prontuario)
        df_oocitos = query_micromanipulacao_oocitos(clinisys_con, prontuario)
        df_tratamentos = query_tratamentos(clinisys_con, prontuario)
        df_congelamentos_embrioes = query_congelamentos_embrioes(clinisys_con, prontuario)
        df_descongelamentos_embrioes = query_descongelamentos_embrioes(clinisys_con, prontuario)
        df_embrioes_congelados = query_embrioes_congelados(clinisys_con, prontuario)
        df_congelamentos_ovulos = query_congelamentos_ovulos(clinisys_con, prontuario)
        df_descongelamentos_ovulos = query_descongelamentos_ovulos(clinisys_con, prontuario)
        df_ovulos_congelados = query_ovulos_congelados(clinisys_con, prontuario)
        
        # Query huntington data lake tables
        df_clinisys_embrioes = query_clinisys_embrioes(huntington_con, prontuario)
        df_embryoscope_embrioes = query_embryoscope_embrioes(huntington_con, prontuario)
        df_embryoscope_clinisys_combined = query_embryoscope_clinisys_combined(huntington_con, prontuario)
        # df_embryoscope_treatment = query_embryoscope_treatment(huntington_con, prontuario)
        df_planilha_embriologia_fet = query_planilha_embriologia_fet(huntington_con, prontuario)
        df_planilha_embriologia_fresh = query_planilha_embriologia_fresh(huntington_con, prontuario)
        df_planilha_embryoscope_combined = query_planilha_embryoscope_combined(huntington_con, prontuario)
        df_redlara_unified = query_redlara_unified(huntington_con, prontuario)
        df_redlara_planilha_combined = query_redlara_planilha_combined(huntington_con, prontuario)

        # Summary
        print(f"\n{'='*80}")
        print(f"QUERY SUMMARY FOR PRONTUARIO {prontuario}")
        print(f"{'='*80}")
        print(f"- view_micromanipulacao: {len(df_micromanipulacao)} records")
        print(f"- view_micromanipulacao_oocitos: {len(df_oocitos)} records")
        print(f"- view_tratamentos: {len(df_tratamentos)} records")
        print(f"- view_congelamentos_embrioes: {len(df_congelamentos_embrioes)} records")
        print(f"- view_descongelamentos_embrioes: {len(df_descongelamentos_embrioes)} records")
        print(f"- view_embrioes_congelados: {len(df_embrioes_congelados)} records")
        print(f"- view_congelamentos_ovulos: {len(df_congelamentos_ovulos)} records")
        print(f"- view_descongelamentos_ovulos: {len(df_descongelamentos_ovulos)} records")
        print(f"- view_ovulos_congelados: {len(df_ovulos_congelados)} records")
        print(f"- gold.clinisys_embrioes: {len(df_clinisys_embrioes)} records")
        print(f"- gold.embryoscope_embrioes: {len(df_embryoscope_embrioes)} records")
        print(f"- gold.embryoscope_clinisys_combined: {len(df_embryoscope_clinisys_combined)} records")
        print(f"- silver.planilha_embriologia_fet: {len(df_planilha_embriologia_fet)} records")
        print(f"- silver.planilha_embriologia_fresh: {len(df_planilha_embriologia_fresh)} records")
        print(f"- gold.planilha_embryoscope_combined: {len(df_planilha_embryoscope_combined)} records")
        print(f"- silver.redlara_unified: {len(df_redlara_unified)} records")
        print(f"- gold.redlara_planilha_combined: {len(df_redlara_planilha_combined)} records")

        print(f"{'='*80}")
        
        # Return results as dictionary
        results = {
            'micromanipulacao': df_micromanipulacao,
            'micromanipulacao_oocitos': df_oocitos,
            'tratamentos': df_tratamentos,
            'congelamentos_embrioes': df_congelamentos_embrioes,
            'descongelamentos_embrioes': df_descongelamentos_embrioes,
            'embrioes_congelados': df_embrioes_congelados,
            'congelamentos_ovulos': df_congelamentos_ovulos,
            'descongelamentos_ovulos': df_descongelamentos_ovulos,
            'ovulos_congelados': df_ovulos_congelados,
            'clinisys_embrioes': df_clinisys_embrioes,
            'embryoscope_embrioes': df_embryoscope_embrioes,
            'embryoscope_clinisys_combined': df_embryoscope_clinisys_combined,
            'planilha_embriologia_fet': df_planilha_embriologia_fet,
            'planilha_embriologia_fresh': df_planilha_embriologia_fresh,
            'planilha_embryoscope_combined': df_planilha_embryoscope_combined,
            'redlara_unified': df_redlara_unified,
            'redlara_planilha_combined': df_redlara_planilha_combined,
        }
        
        return results
        
    finally:
        # Close connections
        clinisys_con.close()
        huntington_con.close()
        print("\nDatabase connections closed.")


## Example Usage


In [40]:
# Example usage
prontuario = 903389

# Query all embryo data
results = query_embryo_data(prontuario)



QUERYING EMBRYO DATA FOR PRONTUARIO: 903389
Query executed at: 2026-02-20 13:27:13.699102

=== VIEW_MICROMANIPULACAO ===
Found 1 records in view_micromanipulacao

=== VIEW_MICROMANIPULACAO_OOCITOS ===
Found 20 records in view_micromanipulacao_oocitos

=== VIEW_TRATAMENTOS ===
Found 1 records in view_tratamentos

=== VIEW_CONGELAMENTOS_EMBRIOES ===
Found 1 records in view_congelamentos_embrioes

=== VIEW_DESCONGELAMENTOS_EMBRIOES ===
Found 0 records in view_descongelamentos_embrioes

=== VIEW_EMBRIOES_CONGELADOS ===
Found 4 records in view_embrioes_congelados

=== VIEW_CONGELAMENTOS_OVULOS ===
Found 0 records in view_congelamentos_ovulos

=== VIEW_DESCONGELAMENTOS_OVULOS ===
Found 0 records in view_descongelamentos_ovulos

=== VIEW_OVULOS_CONGELADOS ===
Found 0 records in view_ovulos_congelados

=== GOLD.CLINISYS_EMBRIOES ===
Found 20 records in gold.clinisys_embrioes

=== GOLD.EMBRYOSCOPE_EMBRIOES ===
Found 4 records in gold.embryoscope_embrioes

=== GOLD.EMBRYOSCOPE_CLINISYS_COMBINED

## Display Results

### 1. Micromanipulacao


In [41]:
print("=== MICROMANIPULACAO DATA ===")
micromanipulacao_data = results['micromanipulacao']
print(f"Records: {len(micromanipulacao_data)}")
if not micromanipulacao_data.empty:
    display(micromanipulacao_data)
else:
    print("No micromanipulacao data found.")


=== MICROMANIPULACAO DATA ===
Records: 1


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,codigo_congelamento_semen,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,abstinencia_recomendada,recepcao_ovulos,CicloDoadora,IdadeDoadora,checagem_d4,Denud_DL,checagem_fertilizacao,checagem_d2,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,sangue_cateter_interno,dificuldade_transf,retorno_embrioes,vezes_retorno,sangue_cateter_externo,ICSIDescongelados,CicloDescongelamento,pgd_oocito,pgd_d3,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,mini_gradiente,data_procedimento_2,tecnica_injetados_2,motilidade_2,morfologia_injetados_2,unidade_id,data_microtese,data_microtese_2,validade7,responsavel_denud_id,profissional_fertilizacao_id,profissional_d2_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,validade11,materiais_insumos,local_coleta,fonte_outros,gradiente_descontinuo_2,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,mini_gradiente_2,local_coleta_2,fonte_outros_2,observacoes_resumo_ciclos,viscosidade_2,volume_amostra_utilizada_2,morfologia_pre_2,morfologia_pos_2,zymot_2,validade20,gradiente,lote21,validade21,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,c_ps1,d_ps1,conc_ps1,diametro_minimo_pse,diametro_medio_pse,diametro_maximo_pse,tubulos_avaliados_pse,peso_total_pse,peso_microtese_pse,peso_biopsia_pse,diametro_minimo_psd,diametro_medio_psd,diametro_maximo_psd,tubulos_avaliados_psd,peso_total_psd,peso_microtese_psd,peso_biopsia_psd,responsavel_transfer_id,MorfologiaAlterada,EstadoSptz,CongelamentoSemenHom,VialsDescongeladasHom,CongelamentoSemenHet,VialsDescongeladasHet,check_gradiente_ps,concentrado,check_swimup,check_lavagem,volume_ps2,MotivoCongelamentoOvulos,concentracao_progressivos_pos,c_ps2,d_ps2,conc_ps2,comentarios_ps,responsavel_ps,fonte2,inicio_ps2,termino_ps2,volume_ps3,concentracao_progressivos_pre_2,c_ps3,d_ps3,conc_ps3,diametro_minimo_pse2,diametro_medio_pse2,diametro_maximo_pse2,tubulos_avaliados_pse2,peso_total_pse2,peso_microtese_pse2,peso_biopsia_pse2,diametro_minimo_psd2,diametro_medio_psd2,diametro_maximo_psd2,tubulos_avaliados_psd2,peso_total_psd2,peso_microtese_psd2,peso_biopsia_psd2,timelapse,EstadoSptz2,codigo_congelamento_semen_2,CongelamentoSemenHom2,VialsDescongeladasHom2,CongelamentoSemenHet2,VialsDescongeladasHet2,check_gradiente_ps2,concentrado_2,check_swimup2,check_lavagem2,volume_ps4,concentracao_progressivos_pos_2,c_ps4,d_ps4,conc_ps4,comentarios_ps2,responsavel_ps2,transferidos,cateter,lote_cateter,validade_cateter,motivo_nao_transferencia,Transfer_D5,medico_responsavel_id,azoospermia,cod1_crio,cod2_crio,tanque,cane,tecnica,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,responsavel_biopsia,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,28387,4098/25,903389,38,38,2025-11-14,PS861,13:10,13:17,09:00,65,3583,None,Não,None,None,None,11:10,07:00,None,07:00,07:00,10:00,None,12,20,None,None,None,None,None,Não,None,None,None,1,2025-11-14,ICSI,15 espermatozoides móveis progressivos\r\n,15 espermatozoides normais\r\n,None,1,None,None,None,None,0000000002,<NA>,<NA>,<NA>,3648,5162,None,3721,

### 2. Micromanipulacao Oocitos


In [42]:
print("=== MICROMANIPULACAO OOCITOS DATA ===")
oocitos_data = results['micromanipulacao_oocitos']
print(f"Records: {len(oocitos_data)}")
if not oocitos_data.empty:
    display(oocitos_data)
else:
    print("No micromanipulacao oocitos data found.")


=== MICROMANIPULACAO OOCITOS DATA ===
Records: 20


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,ComentariosAntes,Embriologista,PI,TCD,AH,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,GD1,OocitoDoado,ICSI,OrigemOocito,InseminacaoOocito,NClivou_D2,NCelulas_D2,Frag_D2,Blastomero_D2,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,GD2,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,ResultadoPGDDetalhes,embryo_number
0,286284,28387,Não,MII,None,None,3583,3648,Criopreservado,None,Sim,Não analisado,None,2PN,C1,Criopreservado,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,8,1,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 5,A,B,None,None,None,None,None,None,None,None,None,None,7.75,Sim,8b48595483ee305600069154356449ab,2026-02-05 21:00:42,None,1
1,286285,28387,Não,MII,None,None,3585,None,Descartado,None,None,None,None,DEG,C1,Criopreservado,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4a8cb101f3c75333e9e74e4a7da5dfe4,2026-02-05 21:00:42,None,2
2,286286,28387,Não,MII,None,None,3583,None,Descartado,None,None,None,None,1CP,C1,Criopreservado,None,Não,None,Fresco,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4e5eab91e8dafcf317358de866d65dc3,2026-02-05 21:00:42,None,3
3,286287,28387,Não,MII,None,None,3583,3648,Criopreservado,None,Sim,Euploide,PGT-A,2PN,C1,Criopreservado,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,12,2,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 5,A,A,None,None,None,None,None,None,None,None,None,None,7.84,Sim,bc671f0d92f6f6b681b6a540327eb326,2026-02-05 21:00:42,Feminino,4
4,286288,28387,Não,MII,None,None,3583,3648,Criopreservado,None,Sim,Não analisado,None,2PN,C1,Criopreservado,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,8,1,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 5,A,A,None,None,None,None,None,None,None,None,None,None,NI,Sim,1a1214628b25956684cfef050b07aed8,2026-02-05 21:00:42,None,5
5,286289,28387,Não,MII,None,None,3583,None,Descartado,None,None,None,None,DEG,C1,Criopreservado,None,Não,None,Fresco,ICSI,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,d5c178a5c4f74f91c1a73fd8d18251b7,2026-02-05 21:00:42,None,6
6,286290,28387,Não,MI > MII,None,None,5162,None,Descartado,None,None,None,None,2PN,C1,Criopreservado,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,5,3,Simétricos,None,None,None,None,None,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,77e678501991864139d299e7d25383a1,2026-02-05 21:00:42,None,7
7,286291,28387,Não,MI > MII,None,None,5162,None,Descartado,None,None,None,None,1CP,C1,Criopreservado,None,Não,None,Fresco,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,f97c88482a0a2aca88135910025c3c59,2026-02-05 21:00:42,None,8
8,286292,28387,Não,MI > MII,None,None,5162,None,Descartado,None,None,None,None,2PN,C1,Criopreservado,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,4,3,Assimétricos,None,None,None,None,None,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,790ad3922e37cbbdc962c47202fcf5b9,2026-02-05 21:00:42,None,9
9,286293,28387,Não,MI > MII,None,None,5162,None,Descartado,None,None,None,None,2PN,C1,Criopreservado,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,10,2,Simétricos,None,None,None,None,None,None,None,None,None,None,None,Não,N

### 3. Tratamentos


In [43]:
print("=== TRATAMENTOS DATA ===")
tratamentos_data = results['tratamentos']
print(f"Records: {len(tratamentos_data)}")
if not tratamentos_data.empty:
    display(tratamentos_data)
else:
    print("No tratamentos data found.")


=== TRATAMENTOS DATA ===
Records: 1


,id,prontuario,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,hora_procedimento,tipo_procedimento,motivo_congelamento,tipo_inducao,protocolo,tipo_ciclo_congelado,tipo_preparo_endometrial,num_congelados,num_transferir,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,material_heterologo,utero_substituicao,prontuario_genitores,prontuario_uter_substituicao,doacao_ovulos,destino_doacao_ovulos,prontuario_receptora,complemento_receptora,doacao_embrioes,era,emma,alice,matrice_lab,imunohistoquimica,outros_biopsia,consentimento,financeiro,embryoscope,pgt,era_auxiliar,recomendacao_transferencia,resultado_era,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,fator_infertilidade3,fsh_exame,foliculos_antrais,data_histeroscopia,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,ropa,data_inicio_agonista,gnrh,fsh_agonista,lh_agonista,outros_medicamentos,gonal,menogon,menopur,pergoveris,elonva,serofene_clomid_indux,luveris,puregon,letrozole,fostimon,merional,rekovelle,descricao_outras_medicacoes,dose_total_fsh,dose_total_lh,dose_total_fsh_lh,coorte_e2,coorte_aco,coorte_progesterona,coq10,hormonal_combinado,coorte_testosterona,agonista_coorte,coorte_antagonista,coorte_dhea,utilizado_hcg,motivo_cancelamento_hcg,data_hcg,horario_planejado_hcg,horario_real_hcg,ovidrel,gonapeptyl,dose_gonapeptyl,outras_complicacoes_hcg,dose_outras_complicacoes,lupron,outros_hcg,descricao_outros_hcg,data_inicio_progesterona,p4_inicio_progesterona,estradiol_fase_lutea,progesterona_fase_lutea,progesterona_oral,progesterona_vaginal,progesterona_injetavel,combinacao_oral_vaginal,nenhuma_progesterona,hcg_progesterona,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,observacoes_descongelamento,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,p4_transferencia,motivo_nao_transferir,motivo_cancelamento_tratamento,ligacao_enfermagem,beta_valor,beta_valor2,data_ultrassom1,data_ultrassom2,num_sacos_gestacionais,tsh,data_ultrassom1_contato,descricao_ultrassom1,bcf_embrião1_status,bcf_embrião1_fonte,data_ultrassom2_contato,descricao_ultrassom2,data_ultrassom3_contato,descricao_ultrassom3,resultado_tratamento,analise_genetica,resultado_analise_genetica,motivo_perda_aborto,data_nascimento_bebes,obstetra,nome_bebe1,sexo_bebe1,peso_bebe1,altura_bebe1,idade_gestacional_bebe1,via_parto_bebe1,viabilidade_bebe1,anomalias_bebe1,nome_bebe2,sexo_bebe2,peso_bebe2,altura_bebe2,idade_gestacional_bebe2,via_parto_bebe2,viabilidade_bebe2,anomalias_bebe2,nome_bebe3,sexo_bebe3,peso_bebe3,altura_bebe3,idade_gestacional_bebe3,via_parto_bebe3,viabilidade_bebe3,anomalias_bebe3,nome_bebe4,sexo_bebe4,peso_bebe4,altura_bebe4,idade_gestacional_bebe4,via_parto_bebe4,viabilidade_bebe4,anomalias_bebe4,observacoes_bebes,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp,bmi,previous_et,previous_et_od,unidade
0,39538,903389,38,38,esposa,1,2025-11-14,None,Ciclo a Fresco FIV,None,Convencional,None,None,Hormonal,None,None,Homólogo,Fresco,NaN,Heterólogo,Sêmen,Criopreservado,None,None,Não,NaN,NaN,Não,None,NaN,None,Não,None,None,None,None,None,None,None,None,None,PGT-A,None,None,None,None,None,Sim,None,Sim,None,64.0,1.68,Não,2025-11-01,NaN,NaN,None,None,None,None,None,None,None,2025-11-03,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Não utilizado,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaT,None,None,None,None,No transfer,NaT,None,None,None,Criopreservação total,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No transfer,Não,None,None,None

### 4. Congelamentos Embrioes


In [44]:
print("=== CONGELAMENTOS EMBRIOES DATA ===")
congelamentos_data = results['congelamentos_embrioes']
print(f"Records: {len(congelamentos_data)}")
if not congelamentos_data.empty:
    display(congelamentos_data)
else:
    print("No congelamentos embrioes data found.")


=== CONGELAMENTOS EMBRIOES DATA ===
Records: 1


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,condicoes_amostra,empresa_transporte,clinica_origem,responsavel_recebimento,responsavel_recebimento_data,responsavel_armazenamento,responsavel_armazenamento_data,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Ovulo,D2,D3,D4,D5,D6,D7,rack,rack2,rack3,rack4,Observacoes,BiologoResponsavel,BiologoFIV,BiologoFIV2,status_financeiro,responsavel_congelamento_d5,responsavel_checagem_d5,responsavel_congelamento_d6,responsavel_checagem_d6,responsavel_congelamento_d7,responsavel_checagem_d7,hash,extraction_timestamp
0,30920,E3234/25,2,903389,esposa,2025-11-19,1900-01-01 15:30:00,4098/25,Não,None,None,None,<NA>,NaT,<NA>,NaT,4,4,M11748,None,None,None,Vitrificação,NaN,NaN,NaN,NaN,1.0,1.0,NaN,None,NaN,NaN,NaN,None,<NA>,Renata Erberelli,Patricia Marchi,Cobrar,3736,3721,3721,3648,<NA>,<NA>,841e716d6453dc8b0b364368c0fcb555,2025-12-08 19:24:17


### 5. Descongelamentos Embrioes


In [45]:
print("=== DESCONGELAMENTOS EMBRIOES DATA ===")
descongelamentos_data = results['descongelamentos_embrioes']
print(f"Records: {len(descongelamentos_data)}")
if not descongelamentos_data.empty:
    display(descongelamentos_data)
else:
    print("No descongelamentos embrioes data found.")


=== DESCONGELAMENTOS EMBRIOES DATA ===
Records: 0
No descongelamentos embrioes data found.


### 6. Embrioes Congelados


In [46]:
print("=== EMBRIOES CONGELADOS DATA ===")
embrioes_congelados_data = results['embrioes_congelados']
print(f"Records: {len(embrioes_congelados_data)}")
if not embrioes_congelados_data.empty:
    display(embrioes_congelados_data)
    # display(embrioes_congelados_data[embrioes_congelados_data['id_descongelamento']!=0])
else:
    print("No embrioes congelados data found.")


=== EMBRIOES CONGELADOS DATA ===
Records: 4


,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,cores,embriao,doado,lote,celulas,qualidade,qualidade_recongelamento,comentarios,comentarios_descongelamento,ah,dia2,maturidade,dia2_2,dia2_3,dia3,dia3_2,dia3_3,dia4,dia4_2,dia4_3,dia4_4,gd,gd2,gd3,dia5,dia5_2,dia5_3,dia5_4,dia6,dia6_2,dia6_3,dia6_4,dia7,dia7_2,dia7_3,dia7_4,clivou_d2,clivou_d3,clivou_d4,clivou_d5,clivou_d6,clivou_d7,transferidos,pgd,resultado_pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,observacao,destino,hash,extraction_timestamp
0,92844,286296,30920,0,903389,#13,M11748,<NA>,#13,None,None,None,Blastocisto Grau 4 - A - B,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,1.4,D6,8.35,A8,4,M11748,None,<NA>,c0e2a45859fb0d8e10768acd5c26946e,2026-01-16 19:36:05
1,92843,286288,30920,0,903389,#5,M11748,<NA>,#5,None,None,None,Blastocisto Grau 5 - A - A,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,8.4,D5,NI,A8,4,M11748,None,<NA>,e61aed417287030ef7c15adc39664c14,2026-01-16 19:36:05
2,92842,286287,30920,0,903389,#4,M11748,<NA>,#4,None,None,None,Blastocisto Grau 5 - A - A,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,Feminino,Sim,Euploide,9.2,D5,7.84,A8,4,M11748,None,<NA>,b7ca91fbe43cf66b0e09398046e9297a,2026-01-16 19:36:05
3,92841,286284,30920,0,903389,#1,M11748,<NA>,#1,None,None,None,Blastocisto Grau 5 - A - B,<NA>,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,6.0,D5,7.75,A8,4,M11748,None,<NA>,e79e6f9fc7a2a7de88da708ad07bdec0,2026-01-16 19:36:05


### 7. Clinisys Embrioes


In [47]:
print("=== CLINISYS EMBRIOES DATA ===")
clinisys_embrioes_data = results['clinisys_embrioes']
print(f"Records: {len(clinisys_embrioes_data)}")
if not clinisys_embrioes_data.empty:
    display(clinisys_embrioes_data.head(30))
else:
    print("No clinisys embrioes data found.")


=== CLINISYS EMBRIOES DATA ===
Records: 20


,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,oocito_ResultadoPGD,oocito_ResultadoPGDDetalhes,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od,nome_medico,tipo_medico,flag_is_external_medical
0,286284,28387,Criopreservado,1,Fresco,Não analisado,None,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,92841,Blastocisto Grau 5 - A - B,None,30920,E3234/25,2025-11-19,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,Layza Borges,Médico Parceiro,1
1,286285,28387,Descartado,2,Fresco,None,None,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,<NA>,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,Layza Borges,Médico Parceiro,1
2,286286,28387,Descartado,3,Fresco,None,None,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,<NA>,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,Layza Borges,Médico Parceiro,1
3,286287,28387,Criopreservado,4,Fresco,Euploide,Feminino,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,92842,Blastocisto Grau 5 - A - A,None,30920,E3234/25,2025-11-19,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,Layza Borges,Médico Parceiro,1
4,286288,28387,Criopreservado,5,Fresco,Não analisado,None,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,92843,Blastocisto Grau 5 - A - A,None,30920,E3234/25,2025-11-19,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,Layza Borges,Médico Parceiro,1
5,286289,28387,Descartado,6,Fresco,None,None,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,<NA>,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,Layza Borges,Médico Parceiro,1
6,286290,28387,Descartado,7,Fresco,None,None,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,<NA>,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,Layza Borges,Médico Parceiro,1
7,286291,28387,Descartado,8,Fresco,None,None,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,<NA>,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,Layza Borges,Médico Parceiro,1
8,286292,28387,Descartado,9,Fresco,None,None,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,<NA>,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,Layza Borges,Médico Parceiro,1
9,286293,28387,Descartado,10,Fresco,None,None,4098/25,903389,2025-11-1

In [48]:
# clinisys_embrioes_data[clinisys_embrioes_data['oocito_id'].isin([184181])]

In [49]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'oocito_id_micromanipulacao', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento'
]

# clinisys_embrioes_data.loc[clinisys_embrioes_data['oocito_embryo_number'].isin([33,35]), colunas]

### 8. Embryoscope Embrioes


In [50]:
print("=== EMBRYOSCOPE EMBRIOES DATA ===")
embryoscope_embrioes_data = results['embryoscope_embrioes']
print(f"Records: {len(embryoscope_embrioes_data)}")
if not embryoscope_embrioes_data.empty:
    display(embryoscope_embrioes_data)
else:
    print("No embryoscope embrioes data found.")


=== EMBRYOSCOPE EMBRIOES DATA ===
Records: 4


,prontuario,patient_PatientID,patient_PatientIDx,patient_FirstName,patient_LastName,patient_DateOfBirth,patient_YearOfBirth,patient_unit_huntington,treatment_TreatmentName,embryo_EmbryoID,embryo_EmbryoDate,embryo_embryo_number,embryo_Position,embryo_InstrumentNumber,embryo_WellNumber,embryo_FertilizationTime,embryo_FertilizationMethod,embryo_EmbryoFate,embryo_Description,embryo_EmbryoDescriptionID,embryo_KIDScore,embryo_KIDDate,embryo_KIDVersion,embryo_KIDUser,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Value_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Value_EVEN2,embryo_Timestamp_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Value_EVEN4,embryo_Timestamp_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Value_EVEN8,embryo_Timestamp_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Value_FRAG2,embryo_Timestamp_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Value_FRAG4,embryo_Timestamp_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Value_FRAG8,embryo_Timestamp_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Value_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Value_ICM,embryo_Timestamp_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Value_Line,embryo_Timestamp_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Value_MN2Type,embryo_Timestamp_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Value_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Value_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Value_PN,embryo_Timestamp_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Value_Pulsing,embryo_Timestamp_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Value_Strings,embryo_Timestamp_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Value_TE,embryo_Timestamp_TE,embryo_Name_t2,embryo_Time_t2,embryo_Value_t2,embryo_Timestamp_t2,embryo_Name_t3,embryo_Time_t3,embryo_Value_t3,embryo_Timestamp_t3,embryo_Name_t4,embryo_Time_t4,embryo_Value_t4,embryo_Timestamp_t4,embryo_Name_t5,embryo_Time_t5,embryo_Value_t5,embryo_Timestamp_t5,embryo_Name_t6,embryo_Time_t6,embryo_Value_t6,embryo_Timestamp_t6,embryo_Name_t7,embryo_Time_t7,embryo_Value_t7,embryo_Timestamp_t7,embryo_Name_t8,embryo_Time_t8,embryo_Value_t8,embryo_Timestamp_t8,embryo_Name_t9,embryo_Time_t9,embryo_Value_t9,embryo_Timestamp_t9,embryo_Name_tB,embryo_Time_tB,embryo_Value_tB,embryo_Timestamp_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Value_tEB,embryo_Timestamp_tEB,embryo_Name_tHB,embryo_Time_tHB,embryo_Value_tHB,embryo_Timestamp_tHB,embryo_Name_tM,embryo_Time_tM,embryo_Value_tM,embryo_Timestamp_tM,embryo_Name_tPB2,embryo_Time_tPB2,embryo_Value_tPB2,embryo_Timestamp_tPB2,embryo_Name_tPNa,embryo_Time_tPNa,embryo_Value_tPNa,embryo_Timestamp_tPNa,embryo_Name_tPNf,embryo_Time_tPNf,embryo_Value_tPNf,embryo_Timestamp_tPNf,embryo_Name_tSB,embryo_Time_tSB,embryo_Value_tSB,embryo_Timestamp_tSB,embryo_Name_tSC,embryo_Time_tSC,embryo_Value_tSC,embryo_Timestamp_tSC,idascore_IDAScore,idascore_IDATime,idascore_IDAVersion,AgeAtFertilization
0,903389,903389,PC1R85KM_45975.4659080787,"CIRINO, DIANA P. O.",15/02/1987,1987-02-15,1987,Vila Mariana,2025 - 2158,D2025.11.14_S03901_I3253_P-5,2025-11-14,4,9,3253,5,2025-11-14 13:10:00,None,Unknown,None,AA5,None,NaT,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,None,None,None,NaN,N

In [51]:
# embryoscope_embrioes_data.loc[embryoscope_embrioes_data['embryo_FertilizationTime'].isin(['2024-01-15 12:45:00'])].sort_values(by='embryo_embryo_number')

### 9. Embryoscope Clinisys Combined


In [52]:
print("=== EMBRYOSCOPE CLINISYS COMBINED DATA ===")
combined_data = results['embryoscope_clinisys_combined']
print(f"Records: {len(combined_data)}")
if not combined_data.empty:
    display(combined_data.head(20))
else:
    print("No embryoscope clinisys combined data found.")


=== EMBRYOSCOPE CLINISYS COMBINED DATA ===
Records: 20


,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,oocito_ResultadoPGD,oocito_ResultadoPGDDetalhes,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od,nome_medico,tipo_medico,flag_is_external_medical,prontuario,patient_PatientID,patient_PatientIDx,patient_FirstName,patient_LastName,patient_DateOfBirth,patient_YearOfBirth,patient_unit_huntington,treatment_TreatmentName,embryo_EmbryoID,embryo_EmbryoDate,embryo_embryo_number,embryo_Position,embryo_InstrumentNumber,embryo_WellNumber,embryo_FertilizationTime,embryo_FertilizationMethod,embryo_EmbryoFate,embryo_Description,embryo_EmbryoDescriptionID,embryo_KIDScore,embryo_KIDDate,embryo_KIDVersion,embryo_KIDUser,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Value_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Value_EVEN2,embryo_Timestamp_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Value_EVEN4,embryo_Timestamp_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Value_EVEN8,embryo_Timestamp_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Value_FRAG2,embryo_Timestamp_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Value_FRAG4,embryo_Timestamp_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Value_FRAG8,embryo_Timestamp_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Value_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Value_ICM,embryo_Timestamp_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Value_Line,embryo_Timestamp_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Value_MN2Type,embryo_Timestamp_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Value_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Value_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Value_PN,embryo_Timestamp_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Value_Pulsing,embryo_Timestamp_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Value_Strings,embryo_Timestamp_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Value_TE,embryo_Timestamp_TE,embryo_Name_t2,embryo_Time_t2,embryo_Value_t2,embryo_Timestamp_t2,embryo_Name_t3,embryo_Time_t3,embryo_Value_t3,embryo_Timestamp_t3,embryo_Name_t4,embryo_Time_t4,embryo_Value_t4,embryo_Timestamp_t4,embryo_Name_t5,embryo_Time_t5,embryo_Value_t5,embryo_Timestamp_t5,embryo_Name_t6,embryo_Time_t6,embryo_Value_t6,embryo_Timestamp_t6,embryo_Name_t7,embryo_Time_t7,embryo_Value_t7,embryo_Timestamp_t7,embryo_Name_t8,embryo_Time_t8,embryo_Value_t8,embryo_Timestamp_t8,embryo_Name_t9,embryo_Time_t9,embryo_Value_t9,embryo_Timestamp_t9,embryo_Name_tB,embryo_Time_tB,embryo_Value_tB,embryo_Timestamp_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Value_tEB,embryo_Timestamp_tEB,embryo_Name_tHB,embry

In [53]:
colunas = [
    'oocito_id','oocito_id_micromanipulacao', 'oocito_embryo_number', 'oocito_TCD', 
    'oocito_id_micromanipulacao', 'micro_numero_caso', 'micro_Data_DL', 'micro_data_procedimento', 
    'cong_em_id', 'cong_em_CodCongelamento', 'cong_em_Ciclo', 'cong_em_NEmbrioes', 
    'treatment_TreatmentName', 'embryo_EmbryoID', 'embryo_KIDScore', 'embryo_Description', 'embryo_EmbryoDescriptionID', 'embryo_embryo_number',
    'emb_cong_transferidos', 'emb_cong_id', 'emb_cong_id_oocito', 'emb_cong_id_congelamento', 'emb_cong_id_descongelamento','emb_cong_embriao', 
    'descong_em_id', 'descong_em_transferidos_transferencia'	, 'descong_em_CodDescongelamento', 'descong_em_Ciclo' , 'descong_em_DataDescongelamento', 'descong_em_DataTransferencia', 
    'trat_id', 'trat_data_procedimento', 'trat_data_transferencia', 'trat_resultado_tratamento', 'trat_tentativa'
]

# combined_data.loc[combined_data['oocito_id_micromanipulacao'].isin([21961]), colunas]
# combined_data.loc[combined_data['embryo_EmbryoDescriptionID'].isin(['AA5']), colunas] 
# combined_data.loc[:, colunas]

### 11. Planilha Embriologia FESH


In [54]:
print("=== PLANILHA EMBRIOLOGIA DATA FRESH===")
planilha_data = results['planilha_embriologia_fresh']
print(f"Records: {len(planilha_data)}")
if not planilha_data.empty:
    display(planilha_data)
else:
    print("No planilha embriologia data found.")


=== PLANILHA EMBRIOLOGIA DATA FRESH===
Records: 1


,pin,tipo_1,tipo_de_inseminacao,tipo_biopsia,data_da_puncao,altura,peso,data_de_nasc,idade_espermatozoide,fator_1,origem,tipo,incubadora,opu,total_de_mii,qtd_blasto,qtd_blasto_tq_a_e_b,qtd_analisados,qtd_normais,dia_cryo,data_crio,no_biopsiados,origem_espermatozoide,tipo_espermatozoide,file_name,sheet_name,prontuario
0,903389,FRESH,ICSI,PGT-A,2025-11-14,168.0,64,1987-02-15,DONOR,NOT AVAILABLE,DONATED,CRYOPRESERVED,ES,20,15,4,4,1,1,5 E 6,2025-11-19,<NA>,None,None,CASOS 2025 VM.xlsx,FRESH,903389


### 12. Planilha Embriologia FET


In [55]:
print("=== PLANILHA EMBRIOLOGIA DATA FET===")
planilha_data = results['planilha_embriologia_fet']
print(f"Records: {len(planilha_data)}")
if not planilha_data.empty:
    display(planilha_data)
else:
    print("No planilha embriologia data found.")


=== PLANILHA EMBRIOLOGIA DATA FET===
Records: 0
No planilha embriologia data found.


### 13. Planilha Embryoscope Combined


In [56]:
print("=== PLANILHA EMBRYOSCOPE COMBINED DATA ===")
combined_planilha_data = results['planilha_embryoscope_combined']
print(f"Records: {len(combined_planilha_data)}")
if not combined_planilha_data.empty:
    display(combined_planilha_data.head(20))
else:
    print("No planilha embryoscope combined data found.")


=== PLANILHA EMBRYOSCOPE COMBINED DATA ===
Records: 20


,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,oocito_ResultadoPGD,oocito_ResultadoPGDDetalhes,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od,nome_medico,tipo_medico,flag_is_external_medical,prontuario,patient_PatientID,patient_PatientIDx,patient_FirstName,patient_LastName,patient_DateOfBirth,patient_YearOfBirth,patient_unit_huntington,treatment_TreatmentName,embryo_EmbryoID,embryo_EmbryoDate,embryo_embryo_number,embryo_Position,embryo_InstrumentNumber,embryo_WellNumber,embryo_FertilizationTime,embryo_FertilizationMethod,embryo_EmbryoFate,embryo_Description,embryo_EmbryoDescriptionID,embryo_KIDScore,embryo_KIDDate,embryo_KIDVersion,embryo_KIDUser,embryo_Name_BlastExpandLast,embryo_Time_BlastExpandLast,embryo_Value_BlastExpandLast,embryo_Timestamp_BlastExpandLast,embryo_Name_BlastomereSize,embryo_Time_BlastomereSize,embryo_Value_BlastomereSize,embryo_Timestamp_BlastomereSize,embryo_Name_EVEN2,embryo_Time_EVEN2,embryo_Value_EVEN2,embryo_Timestamp_EVEN2,embryo_Name_EVEN4,embryo_Time_EVEN4,embryo_Value_EVEN4,embryo_Timestamp_EVEN4,embryo_Name_EVEN8,embryo_Time_EVEN8,embryo_Value_EVEN8,embryo_Timestamp_EVEN8,embryo_Name_FRAG2,embryo_Time_FRAG2,embryo_Value_FRAG2,embryo_Timestamp_FRAG2,embryo_Name_FRAG2CAT,embryo_Time_FRAG2CAT,embryo_Value_FRAG2CAT,embryo_Timestamp_FRAG2CAT,embryo_Name_FRAG4,embryo_Time_FRAG4,embryo_Value_FRAG4,embryo_Timestamp_FRAG4,embryo_Name_FRAG8,embryo_Time_FRAG8,embryo_Value_FRAG8,embryo_Timestamp_FRAG8,embryo_Name_Fragmentation,embryo_Time_Fragmentation,embryo_Value_Fragmentation,embryo_Timestamp_Fragmentation,embryo_Name_ICM,embryo_Time_ICM,embryo_Value_ICM,embryo_Timestamp_ICM,embryo_Name_Line,embryo_Time_Line,embryo_Value_Line,embryo_Timestamp_Line,embryo_Name_MN2Type,embryo_Time_MN2Type,embryo_Value_MN2Type,embryo_Timestamp_MN2Type,embryo_Name_MorphologicalGrade,embryo_Time_MorphologicalGrade,embryo_Value_MorphologicalGrade,embryo_Timestamp_MorphologicalGrade,embryo_Name_MultiNucleation,embryo_Time_MultiNucleation,embryo_Value_MultiNucleation,embryo_Timestamp_MultiNucleation,embryo_Name_Nuclei2,embryo_Time_Nuclei2,embryo_Value_Nuclei2,embryo_Timestamp_Nuclei2,embryo_Name_PN,embryo_Time_PN,embryo_Value_PN,embryo_Timestamp_PN,embryo_Name_Pulsing,embryo_Time_Pulsing,embryo_Value_Pulsing,embryo_Timestamp_Pulsing,embryo_Name_ReexpansionCount,embryo_Time_ReexpansionCount,embryo_Value_ReexpansionCount,embryo_Timestamp_ReexpansionCount,embryo_Name_Strings,embryo_Time_Strings,embryo_Value_Strings,embryo_Timestamp_Strings,embryo_Name_TE,embryo_Time_TE,embryo_Value_TE,embryo_Timestamp_TE,embryo_Name_t2,embryo_Time_t2,embryo_Value_t2,embryo_Timestamp_t2,embryo_Name_t3,embryo_Time_t3,embryo_Value_t3,embryo_Timestamp_t3,embryo_Name_t4,embryo_Time_t4,embryo_Value_t4,embryo_Timestamp_t4,embryo_Name_t5,embryo_Time_t5,embryo_Value_t5,embryo_Timestamp_t5,embryo_Name_t6,embryo_Time_t6,embryo_Value_t6,embryo_Timestamp_t6,embryo_Name_t7,embryo_Time_t7,embryo_Value_t7,embryo_Timestamp_t7,embryo_Name_t8,embryo_Time_t8,embryo_Value_t8,embryo_Timestamp_t8,embryo_Name_t9,embryo_Time_t9,embryo_Value_t9,embryo_Timestamp_t9,embryo_Name_tB,embryo_Time_tB,embryo_Value_tB,embryo_Timestamp_tB,embryo_Name_tEB,embryo_Time_tEB,embryo_Value_tEB,embryo_Timestamp_tEB,embryo_Name_tHB,embry

In [57]:
cols = [
    'oocito_id', 'oocito_id_micromanipulacao', 'oocito_TCD', 'oocito_embryo_number', 
    'oocito_OrigemOocito', 'micro_numero_caso', 'micro_prontuario', 'micro_Data_DL', 
    'micro_data_procedimento', 'micro_oocitos', 'trat1_id', 'trat1_tentativa', 
    'trat1_tipo_procedimento', 'trat1_data_inicio_inducao', 'trat1_motivo_nao_transferir', 
    'trat1_data_transferencia', 'trat1_resultado_tratamento', 'trat1_fator_infertilidade1', 
    'trat1_origem_ovulo', 'trat1_bmi', 'trat1_previous_et', 'trat1_previous_et_od', 
    'emb_cong_id', 'emb_cong_qualidade', 'emb_cong_transferidos', 'cong_em_id', 
    'cong_em_CodCongelamento', 'cong_em_Data', 'descong_em_id', 'descong_em_CodDescongelamento', 
    'descong_em_DataTransferencia', 'descong_em_DataCongelamento', 'descong_em_DataDescongelamento', 
    'trat2_id', 'trat2_tentativa', 'trat2_tipo_procedimento', 'trat2_data_inicio_inducao', 
    'trat2_motivo_nao_transferir', 'trat2_data_transferencia', 'trat2_resultado_tratamento', 
    'trat2_bmi', 'trat2_previous_et', 'trat2_previous_et_od', 'prontuario', 
    'patient_PatientID', 'patient_PatientIDx', 'join_step', 'matched_planilha_prontuario', 
    'matched_planilha_transfer_date', 'merged_numero_de_nascidos', 'incubadora_padronizada', 
    'outcome', 'outcome_type', 'chart_or_pin', 'date_of_delivery', 
    'gestational_age_at_delivery', 'type_of_delivery', 'baby_1_weight', 'baby_2_weight', 
    'baby_3_weight', 'complications_of_pregnancy_specify', 'number_of_fet_after_originally_frozen', 
    'n_of_biopsied', 'n_of_normal', 'number_of_embryos_transferred', 'year', 'unidade', 
    'fresh_fator_1', 'fresh_incubadora', 'fresh_data_crio', 'fresh_tipo_1', 
    'fresh_tipo_de_inseminacao', 'fresh_tipo_biopsia', 'fresh_altura', 'fresh_peso', 
    'fresh_data_de_nasc', 'fresh_idade_espermatozoide', 'fresh_origem', 'fresh_tipo', 
    'fresh_opu', 'fresh_total_de_mii', 'fresh_qtd_blasto', 'fresh_qtd_blasto_tq_a_e_b', 
    'fresh_no_biopsiados', 'fresh_qtd_analisados', 'fresh_qtd_normais', 'fresh_dia_cryo', 
    'fresh_file_name', 'fresh_sheet_name', 'fet_data_crio', 'fet_resultado', 
    'fet_tipo_resultado', 'fet_tipo_1', 'fet_tipo_de_tratamento', 'fet_tipo_de_fet', 
    'fet_tipo_biopsia', 'fet_tipo_da_doacao', 'fet_idade_mulher', 'fet_idade_do_cong_de_embriao', 
    'fet_preparo_para_transferencia', 'fet_dia_cryo', 'fet_no_da_transfer_1a_2a_3a', 
    'fet_dia_et', 'fet_no_et', 'fet_gravidez_bioquimica', 'fet_gravidez_clinica', 
    'fet_file_name', 'fet_sheet_name'
]

combined_planilha_data[cols]

,oocito_id,oocito_id_micromanipulacao,oocito_TCD,oocito_embryo_number,oocito_OrigemOocito,micro_numero_caso,micro_prontuario,micro_Data_DL,micro_data_procedimento,micro_oocitos,trat1_id,trat1_tentativa,trat1_tipo_procedimento,trat1_data_inicio_inducao,trat1_motivo_nao_transferir,trat1_data_transferencia,trat1_resultado_tratamento,trat1_fator_infertilidade1,trat1_origem_ovulo,trat1_bmi,trat1_previous_et,trat1_previous_et_od,emb_cong_id,emb_cong_qualidade,emb_cong_transferidos,cong_em_id,cong_em_CodCongelamento,cong_em_Data,descong_em_id,descong_em_CodDescongelamento,descong_em_DataTransferencia,descong_em_DataCongelamento,descong_em_DataDescongelamento,trat2_id,trat2_tentativa,trat2_tipo_procedimento,trat2_data_inicio_inducao,trat2_motivo_nao_transferir,trat2_data_transferencia,trat2_resultado_tratamento,trat2_bmi,trat2_previous_et,trat2_previous_et_od,prontuario,patient_PatientID,patient_PatientIDx,join_step,matched_planilha_prontuario,matched_planilha_transfer_date,merged_numero_de_nascidos,incubadora_padronizada,outcome,outcome_type,chart_or_pin,date_of_delivery,gestational_age_at_delivery,type_of_delivery,baby_1_weight,baby_2_weight,baby_3_weight,complications_of_pregnancy_specify,number_of_fet_after_originally_frozen,n_of_biopsied,n_of_normal,number_of_embryos_transferred,year,unidade,fresh_fator_1,fresh_incubadora,fresh_data_crio,fresh_tipo_1,fresh_tipo_de_inseminacao,fresh_tipo_biopsia,fresh_altura,fresh_peso,fresh_data_de_nasc,fresh_idade_espermatozoide,fresh_origem,fresh_tipo,fresh_opu,fresh_total_de_mii,fresh_qtd_blasto,fresh_qtd_blasto_tq_a_e_b,fresh_no_biopsiados,fresh_qtd_analisados,fresh_qtd_normais,fresh_dia_cryo,fresh_file_name,fresh_sheet_name,fet_data_crio,fet_resultado,fet_tipo_resultado,fet_tipo_1,fet_tipo_de_tratamento,fet_tipo_de_fet,fet_tipo_biopsia,fet_tipo_da_doacao,fet_idade_mulher,fet_idade_do_cong_de_embriao,fet_preparo_para_transferencia,fet_dia_cryo,fet_no_da_transfer_1a_2a_3a,fet_dia_et,fet_no_et,fet_gravidez_bioquimica,fet_gravidez_clinica,fet_file_name,fet_sheet_name
0,286303,28387,Descartado,20,Fresco,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,<NA>,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,<NA>,<NA>,None,1,903389,NaT,<NA>,Embryoscope,None,None,None,NaT,None,None,None,None,None,None,None,None,None,None,<NA>,None,NOT AVAILABLE,ES,2025-11-19,FRESH,ICSI,PGT-A,168.0,64,1987-02-15,DONOR,DONATED,CRYOPRESERVED,20,15,4,4,<NA>,1,1,5 E 6,CASOS 2025 VM.xlsx,FRESH,NaT,None,None,None,None,None,None,None,<NA>,NaN,None,None,None,<NA>,<NA>,None,None,None,None
1,286302,28387,Descartado,19,Fresco,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,<NA>,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,<NA>,<NA>,None,1,903389,NaT,<NA>,Embryoscope,None,None,None,NaT,None,None,None,None,None,None,None,None,None,None,<NA>,None,NOT AVAILABLE,ES,2025-11-19,FRESH,ICSI,PGT-A,168.0,64,1987-02-15,DONOR,DONATED,CRYOPRESERVED,20,15,4,4,<NA>,1,1,5 E 6,CASOS 2025 VM.xlsx,FRESH,NaT,None,None,None,None,None,None,None,<NA>,NaN,None,None,None,<NA>,<NA>,None,None,None,None
2,286301,28387,Descartado,18,Fresco,4098/25,903389,2025-11-14,2025-11-14,20,39538,1,Ciclo a Fresco FIV,2025-11-03,Criopreservação total,NaT,No transfer,None,Homólogo,22.68,0,0,<NA>,None,None,<NA>,None,NaT,<NA>,None,NaT,NaT,NaT,<NA>,None,None,NaT,None,NaT,None,NaN,<NA>,<NA>,<NA>,<NA>,None,1,903389,NaT,<NA>,Embryoscope,None,None,None,NaT,None,None,None,None,None,None,None,None,None,None,<NA>,None,NOT AVAILABLE,ES,2025-11-19,FRESH,ICSI,PGT-A,168.0,64,1987-02-15,DONOR,DONATED,CRYOPRESERVED,20,15,4,4,<NA>,1,1,5 E 6,CASOS 2025 VM.xlsx,FRESH,NaT,None,None,None,None,None,None,None,<NA>,NaN,None,None,None,<NA>,<NA>,None,None,None,None
3,286300,28387,Descartado,17,Fresco,4098/25,903389,2025-11-14,20

### 14. Congelamentos Ovulos


In [58]:
print("=== CONGELAMENTOS OVULOS DATA ===")
congelamentos_ovulos_data = results['congelamentos_ovulos']
print(f"Records: {len(congelamentos_ovulos_data)}")
if not congelamentos_ovulos_data.empty:
    display(congelamentos_ovulos_data)
else:
    print("No congelamentos ovulos data found.")


=== CONGELAMENTOS OVULOS DATA ===
Records: 0
No congelamentos ovulos data found.


### 15. Descongelamentos Ovulos


In [59]:
print("=== DESCONGELAMENTOS OVULOS DATA ===")
descongelamentos_ovulos_data = results['descongelamentos_ovulos']
print(f"Records: {len(descongelamentos_ovulos_data)}")
if not descongelamentos_ovulos_data.empty:
    display(descongelamentos_ovulos_data)
else:
    print("No descongelamentos ovulos data found.")


=== DESCONGELAMENTOS OVULOS DATA ===
Records: 0
No descongelamentos ovulos data found.


### 16. Ovulos Congelados


In [60]:
print("=== OVULOS CONGELADOS DATA ===")
ovulos_congelados_data = results['ovulos_congelados']
print(f"Records: {len(ovulos_congelados_data)}")
if not ovulos_congelados_data.empty:
    display(ovulos_congelados_data)
else:
    print("No ovulos congelados data found.")


=== OVULOS CONGELADOS DATA ===
Records: 0
No ovulos congelados data found.


### 17. Redlara 


In [61]:
print("=== Redlara Unified ===")
redlara_unified = results['redlara_unified']
print(f"Records: {len(redlara_unified)}")
if not redlara_unified.empty:
    display(redlara_unified)
else:
    print("No redlara unified data found.")


=== Redlara Unified ===
Records: 0
No redlara unified data found.


### 18. Redlara - planilha combined


In [62]:
print("=== Redlara planilha combined ===")
redlara_planilha_combined = results['redlara_planilha_combined']
print(f"Records: {len(redlara_planilha_combined)}")
if not redlara_planilha_combined.empty:
    display(redlara_planilha_combined)
else:
    print("No redlara_planilha_combined data found.")

=== Redlara planilha combined ===
Records: 1


,prontuario,transfer_date,merged_numero_de_nascidos,incubadora_padronizada,outcome,outcome_type,chart_or_pin,date_when_embryos_were_cryopreserved,date_of_delivery,gestational_age_at_delivery,type_of_delivery,baby_1_weight,baby_2_weight,baby_3_weight,complications_of_pregnancy_specify,number_of_fet_after_originally_frozen,n_of_biopsied,n_of_normal,number_of_embryos_transferred,year,unidade,fresh_data_da_puncao,fresh_fator_1,fresh_incubadora,fresh_data_crio,fresh_tipo_1,fresh_tipo_de_inseminacao,fresh_tipo_biopsia,fresh_altura,fresh_peso,fresh_data_de_nasc,fresh_idade_espermatozoide,fresh_origem,fresh_tipo,fresh_opu,fresh_total_de_mii,fresh_qtd_blasto,fresh_qtd_blasto_tq_a_e_b,fresh_no_biopsiados,fresh_qtd_analisados,fresh_qtd_normais,fresh_dia_cryo,fresh_file_name,fresh_sheet_name,fet_data_crio,fet_resultado,fet_tipo_resultado,fet_tipo_1,fet_tipo_de_tratamento,fet_tipo_de_fet,fet_tipo_biopsia,fet_tipo_da_doacao,fet_idade_mulher,fet_idade_do_cong_de_embriao,fet_preparo_para_transferencia,fet_dia_cryo,fet_no_da_transfer_1a_2a_3a,fet_dia_et,fet_no_et,fet_gravidez_bioquimica,fet_gravidez_clinica,fet_file_name,fet_sheet_name
0,903389,NaT,<NA>,Embryoscope,None,None,None,NaT,NaT,None,None,None,None,None,None,None,None,None,None,<NA>,None,2025-11-14,NOT AVAILABLE,ES,2025-11-19,FRESH,ICSI,PGT-A,168.0,64,1987-02-15,DONOR,DONATED,CRYOPRESERVED,20,15,4,4,<NA>,1,1,5 E 6,CASOS 2025 VM.xlsx,FRESH,NaT,None,None,None,None,None,None,None,<NA>,NaN,None,None,None,<NA>,<NA>,None,None,None,None
